In [1]:
# Install dependecies

!pip install transformers  # Library for transformer-based models like BERT, RoBERTa
!pip install datasets  # Library for accessing and managing datasets for NLP tasks
!pip install evaluate  # Library for evaluating model performance on various tasks
!pip install accelerate  # Library for accelerating training and inference with PyTorch
!pip install peft  # Pretrained Error-Fix Transformer (PEFT) for improving language models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [2]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_dataset


# Define model names
roberta_model_name = 'roberta-base-peft'  # PEFT stands for Pretrained Error-Fix Transformer
modified_base = 'roberta-base-modified'  # A modified version of the RoBERTa base model
base_model = 'roberta-base'  # The original RoBERTa base model

# Load dataset
dataset = load_dataset('ag_news')  # Load the AG News dataset

# Initialize tokenizer using the base RoBERTa model
tokenizer = RobertaTokenizer.from_pretrained(base_model)

# Define a preprocessing function
def preprocess(examples):
    # Tokenize the text data, truncating and padding sequences
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized

# Apply preprocessing to the entire dataset in batches, removing the 'text' column
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])

# Split the tokenized dataset into train, evaluation, and test datasets
train_dataset = tokenized_dataset['train']  # Training dataset
eval_dataset = tokenized_dataset['test'].shard(num_shards=2, index=0)  # Evaluation dataset (1st shard)
test_dataset = tokenized_dataset['test'].shard(num_shards=2, index=1)  # Test dataset (2nd shard)


# Extract the number of classess and their names
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [ ]:

# Define training arguments for model training
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the model and training results
    evaluation_strategy='steps',  # Evaluate on the validation set every 'steps' training steps
    learning_rate=5e-5,  # Initial learning rate for the optimizer
    num_train_epochs=1,  # Number of times to iterate over the entire training dataset
    per_device_train_batch_size=16,  # Batch size for each GPU during training
)
# Define a function to create a Trainer object for model training
def get_trainer(model):
    # Create a Trainer object with the specified arguments
    return Trainer(
        model=model,  # The model to be trained
        args=training_args,  # Training arguments for configuring the training process
        train_dataset=train_dataset,  # Training dataset
        eval_dataset=eval_dataset,  # Evaluation dataset
        data_collator=data_collator,  # Data collator for batch processing
    )
# Create a Trainer object for full fine-tuning of the model
full_finetuning_trainer = get_trainer(
    AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label),
)
# Start training the model using the Trainer object
full_finetuning_trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: r3shamsolomon (r3shamsolomon-rediet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,0.442000,0.349161


In [ ]:
# Creating a model for sequence classification based on a pretrained model
model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

# Configuring Lora parameters for PEFT (Proxy Experience Finetuning)
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)

# Applying Lora parameters to the model to create the PEFT model
peft_model = get_peft_model(model, peft_config)

# Printing information about the PEFT model's trainable parameters
print('PEFT Model')
peft_model.print_trainable_parameters()

# Obtaining a trainer object for Lora finetuning
peft_lora_finetuning_trainer = get_trainer(peft_model)

# Initiating the training process for the PEFT model using Lora finetuning
peft_lora_finetuning_trainer.train()

In [ ]:
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer


# LOAD the Saved PEFT model

inference_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label)
tokenizer = AutoTokenizer.from_pretrained(modified_base)

def classify(text):
    # Tokenize the input text, truncate if needed, pad sequences, and convert to PyTorch tensors
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    # Pass the tokenized input through the inference model to get predictions
    output = inference_model(**inputs)

    # Extract the predicted class index from the model output
    prediction = output.logits.argmax(dim=-1).item()

    print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')

In [ ]:
from peft import AutoPeftModelForSequenceClassification from transformers import AutoTokenizer

#LOAD the Saved PEFT model
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label) tokenizer = AutoTokenizer.from_pretrained(modified_base)

def classify(text):
  # Tokenize the input text, truncate if needed, pad sequences, and convert to PyTorch tensors
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
  # Pass the tokenized input through the inference model to get predictions
  output = inference_model(**inputs)

  # Extract the predicted class index from the model output
  prediction = output.logits.argmax(dim=-1).item()

  print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')

In [ ]:

classify( "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")

In [ ]:
from torch.utils.data import DataLoader  # DataLoader for creating data iterators
import evaluate  # Import module for computing evaluation metrics
from tqdm import tqdm  # tqdm for progress tracking during evaluation

# Load the evaluation metric (e.g., accuracy) from the evaluate module
metric = evaluate.load('accuracy')

# Define a function to evaluate the model on a given dataset
def evaluate_model(inference_model, dataset):
    # Create a DataLoader for the evaluation dataset with batch size 8 and using data_collator for batch processing
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    # Determine the device (GPU or CPU) for model inference
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the inference model to the appropriate device (GPU or CPU)
    inference_model.to(device)
    # Set the model in evaluation mode (no gradient computation)
    inference_model.eval()

    # Iterate over batches in the evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # Move the batch to the device (GPU or CPU)
        batch.to(device)
        # Disable gradient calculation for inference
        with torch.no_grad():
            # Perform inference using the inference model on the batch
            outputs = inference_model(**batch)
        # Extract the predicted class indices from the model outputs
        predictions = outputs.logits.argmax(dim=-1)
        # Assign predictions and reference labels to variables for metric computation
        predictions, references = predictions, batch["labels"]
        # Add the batch of predictions and references to the evaluation metric
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    # Compute the evaluation metric (e.g., accuracy) based on added batches
    eval_metric = metric.compute()
    # Print the computed evaluation metric
    print(eval_metric)